## Building a router agent to classify text/reviews and draft up a response

In [1]:
# built-in libraries
import os
os.environ['PYTHONUTF8'] = '1'
from typing import TypeVar, Any

# litellm libraries
import litellm
from litellm.types.utils import ModelResponse, Message
from litellm import completion
from instructor import from_litellm, Mode

# misc libraries
from pydantic import BaseModel, create_model

In [2]:
# built-in libraries
from typing import Literal, TypedDict,  Any, Optional, Tuple, List, Dict, Union

# langgraph libraries
from langgraph.graph import StateGraph, START, END
from langchain_core.runnables.graph import  MermaidDrawMethod

# misc libraries
from pydantic import Field
from IPython.display import Image
import pandas as pd
import numpy as np
#from decouple import config
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from tqdm import tqdm
from sklearn.metrics import f1_score


In [3]:

# local modules
from src.llm import LLMCaller

In [4]:
from datasets import load_dataset
from typing import List, Dict
from nltk.tokenize import sent_tokenize
from collections import defaultdict

import nltk
#nltk.download('punkt') - only needed once

c:\Users\Valdemar Schultz\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer

Watsonx credentials

In [6]:
WX_API_KEY = os.getenv("WX_API_KEY")
WX_PROJECT_ID_RAG = os.getenv("WX_PROJECT_ID_RAG")
WX_API_URL = "https://us-south.ml.cloud.ibm.com"



Getting our LLM caller class - Defined in LLM.py

In [39]:

model = LLMCaller(
    api_key=WX_API_KEY,
    project_id=WX_PROJECT_ID_RAG,
    api_url=WX_API_URL,
    model_id="watsonx/mistralai/mistral-large",
    params = {  
    GenParams.TEMPERATURE: 0.0,
    GenParams.MAX_NEW_TOKENS: 50
    }
)

### Creating our few shot examples data set for the LLM to learn from
We will use the few shot examples to train the LLM to classify the text and draft a response

In [8]:
# 1) Load your CSV
csv_path = os.path.join("data", "ekman_train.csv")
# Expect columns: text, ekman_emotion
df = pd.read_csv(csv_path)
#remove all emojis from the text column
df['text'] = df['text'].str.replace(r'[^\x00-\x7F]+', '', regex=True)


creating few shot dataframe

In [ ]:
# Define the order of emotion columns. idxmax will pick the first one if multiple emotions are present.
emotion_columns = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

# Find the emotion for each row.
# This creates a new series where each value is the column name of the first '1' found in emotion_columns.
df['ekman_emotion'] = df[emotion_columns].idxmax(axis=1)

# Recreate the DataFrame with only the 'text' and 'ekman_emotion' columns.
# This overwrites the original df, which is expected by the subsequent cells.
df = df[['text', 'ekman_emotion']]

# You can print the head of the modified DataFrame to verify
print("Transformed DataFrame head:")
print(df.head())

Transformed DataFrame head:
                                                text ekman_emotion
0                        [NAME] good one i like that           joy
1                   Thats actually interesting to me           joy
2  Why is this getting downvoted. I love Meepo so...       sadness
3                    I'm not offended. Just curious.      surprise
4  We have reached the stage where below 2 millio...       neutral


Createing my embedding tool

In [13]:
# 2. Load a local embedding model
#embed_model = SentenceTransformer("all-MiniLM-L6-v2")
embed_model = SentenceTransformer("msmarco-distilbert-base-v4")


# 3. Helper to embed one or more texts
def embed(texts):
    """
    texts: a single string or list of strings
    returns: numpy array of shape (n_texts,)
    """
    # If a single string, wrap it in a list for the model call
    single = False
    if isinstance(texts, str):
        texts = [texts]
        single = True

    embs = embed_model.encode(texts, convert_to_numpy=True)

    # If only one input, return its vector directly
    return embs[0] if single else embs


Chunking the few-shot examples

In [14]:

# 2) Chunk long texts into 1–2 sentence chunks (if you want)
def chunk_text(text, max_sentences=2):
    sents = sent_tokenize(text)
    for i in range(0, len(sents), max_sentences):
        yield " ".join(sents[i : i + max_sentences]).strip()

shots = []
for _, row in df.iterrows():
    text = row["text"].strip()
    emo  = row["ekman_emotion"]
    for chunk in chunk_text(text):
        shots.append({"text": chunk, "emotion": emo})

# 4) Balance to N examples per emotion (to avoid over-representing any one)
N = 800
by_emo = defaultdict(list)
for shot in shots:
    by_emo[shot["emotion"]].append(shot)

few_shots = []
for emo, lst in by_emo.items():
    few_shots.extend(lst[:N])

print(f"Prepared {len(few_shots)} few-shot examples "
      f"({len(by_emo)} emotions × up to {N} each)")

Prepared 5389 few-shot examples (7 emotions × up to 800 each)


Embedding the few-shot examples

In [15]:
# 1) Embed each shot in place
for shot in few_shots:
    shot["embedding"] = embed(shot["text"])

# 2) (Optional) Build an (N, D) matrix for retrieval
emb_matrix = np.vstack([shot["embedding"] for shot in few_shots]).astype("float32")

print(f"Embedded {len(few_shots)} few-shot examples into a {emb_matrix.shape} array")


Embedded 5389 few-shot examples into a (5389, 768) array


In [29]:
# Define your state type
class ReviewState(TypedDict):
    review_text: str
    emotion_definitions: List[Dict[str, str]]
    few_shot_examples: List[Dict[str, str]] 
    emotion: Optional[str]
    draft_reply: Optional[str]
    verbose: bool = False

Creating the nodes of the system

In [17]:
few_shots: List[Dict[str, object]] = few_shots # List of few-shot examples
emb_matrix: np.ndarray = np.vstack([shot["embedding"] for shot in few_shots]).astype("float32")

defineing the Agent functions 

In [ ]:
def read_review(state: ReviewState) -> dict:
    """Reviewbot loads and logs the incoming customer review"""
    review = state["review_text"]
    if state["verbose"]:
        print(f"Reviewbot received review: “{review}”")
    # no changes to state here
    return {}

#def fetch emotion definitions
def fetch_emotion_definitions(state: ReviewState) -> dict:
    """Fetch Ekman emotion definitions"""
    # Define the Ekman emotions and their definitions
    emotion_definitions = {
        "anger": "A strong feeling of displeasure or hostility.",
        "disgust": "A strong feeling of aversion or repulsion.",
        "fear": "An unpleasant emotion caused by the belief that someone or something is dangerous.",
        "joy": "A feeling of great pleasure and happiness.",
        "sadness": "A state of emotional pain or unhappiness.",
        "surprise": "A brief emotional state resulting from an unexpected event.",
        "neutral": "Lacking strong emotion; neither positive nor negative."
    }
    
    # Store the definitions in the state
    state["emotion_definitions"] = emotion_definitions
    return {"emotion_definitions": emotion_definitions}

#dynamic few-shotting technique, matches unseen reviews to the most similar few-shot examples
def retrieve_few_shot_examples(state: ReviewState) -> dict:
    """
    Embed the incoming review, compute cosine similarity against your pre-embedded few_shots,
    and stash the top-5 (text, emotion) pairs into state['few_shot_examples'].
    """
    # 1) Embed the new review
    q_emb = embed(state["review_text"]).reshape(1, -1).astype("float32")
    
    # 2) Compute cosine similarities
    sims = cosine_similarity(q_emb, emb_matrix)[0]  # shape (N,)
    
    # 3) Grab top-N indices
    topk = sims.argsort()[-7:][::-1]
    
    new_examples = []
    for i in topk:
        if not (0 <= i < len(few_shots)):
            # Optionally, log a warning here if an index is out of bounds,
            # though with argsort this shouldn't happen if emb_matrix and few_shots are aligned.
            # print(f"Warning: Index i={i} is out of bounds for few_shots (len={len(few_shots)})")
            continue

        current_shot_element = few_shots[i]

        if not isinstance(current_shot_element, dict):
            # Optionally, log an error or warning here
            # print(f"Warning: few_shots[{i}] is not a dictionary! It is a {type(current_shot_element)}")
            continue

        try:
            text_val = current_shot_element["text"]
            emotion_val = current_shot_element["emotion"]
            
            example_to_add = {"text": text_val, "emotion": emotion_val}
            new_examples.append(example_to_add)

        except KeyError as e:
            # Optionally, log this error if it's critical
            # print(f"Error: KeyError occurred while processing: {current_shot_element}. Key: {e.args[0]}")
            # Depending on desired robustness, you might skip this example or re-raise
            continue # Skip this example if a key is missing
        except Exception as e:
            # Optionally, log other unexpected errors
            # print(f"Error: An unexpected error occurred while processing: {current_shot_element}. Error: {e}")
            continue # Skip this example on other errors
            
    return {"few_shot_examples": new_examples}

def classify_emotion(state: ReviewState) -> dict:
    """Use the LLM with few-shot context"""
    definitions = state["emotion_definitions"]
    text = state["review_text"]
    # Format few-shot context
    example_lines = "\n".join(
        f"- “{ex['text']}” → {ex['emotion']}"
        for ex in state["few_shot_examples"]
        
    )
    #print(f"Few-shot examples: {example_lines}") used for testing
    prompt = f"""
You are a highly precise emotion-classification agent that excels at detecting emotion from text. 
you will be given a text / review and similar examples with labels 

Your task is to read a text / customer review, understand its content, and assign **one or more** of these emotions: 
anger, disgust, fear, joy, sadness, surprise, or neutral if no clear emotion is detected. 

the text:
\"\"\"{text}\"\"\"

to better help you classify below are the definitions of each possible emotion:
{definitions}

and here are a few similar examples with Labels, to help you classify the above text / review:
{example_lines}

Now classify the text / review into one or more of:
Anger, disgust, fear, joy, sadness, surprise or neutral if no clear emotion is detected.

Respond **only** in JSON with:
  "emotion": < a string or list of strings from the set above.
"""
    response_model = model.create_response_model(
        "FewShotEmotionClassification",
        {
            "emotion": (
                List[Literal["anger","disgust","fear","joy","sadness","surprise","neutral"]],
                Field(description="The chosen Ekman emotion(s)")
            ),
        }
    )
    resp = model.invoke(prompt, response_model=response_model)
    return {"emotion": resp.emotion}

def draft_response(state: ReviewState) -> dict:
    """Draft an empathetic reply based on the classified emotion"""
    text = state["review_text"]
    emo  = state["emotion"]
    prompt = f"""A customer expresses **{emo}** in this review:
\"\"\"{text}\"\"\"
You are a customer-service specialist with excellent empathy and brand voice consistency.  
Given a review and its classified emotion(s), draft a short reply that:

  - Acknowledges their {emo}, WITHOUT writing the specific emotions in the response,
  - Addresses any concerns they raise,
  - Invites further dialogue if needed.
"""
    resp = model.invoke(prompt)
    return {"draft_reply": resp.answer}

#showing all the states in the graph
def notify_user(state: ReviewState) -> dict:
    """Present the final draft back to the support employee"""
    if state["verbose"]:
        print("\n" + "="*40)
        print(f"review : {state["review_text"]}")
        print(f"Review classified as: {state['emotion']}")
        print("Response:")
        print(state["draft_reply"])
        print("="*40 + "\n")
    return {}

No need for conditional edge as it is a linear system

### Creating Graph of the system

In [33]:
# Create the graph
review_graph = StateGraph(ReviewState)  # Initialize with your ReviewState

# Add nodes
review_graph.add_node("read review",       read_review)
review_graph.add_node("fetch emotion definitions", fetch_emotion_definitions)
review_graph.add_node("retrieve few-shot examples", retrieve_few_shot_examples)
review_graph.add_node("classify emotion",  classify_emotion)
review_graph.add_node("draft response",    draft_response)
review_graph.add_node("notify user",      notify_user)

# Wire up the edges
# 1) START → read_review
review_graph.add_edge(START, "read review")

# 2) read_review → fetch_emotion_definitions
review_graph.add_edge("read review", "fetch emotion definitions")

review_graph.add_edge("fetch emotion definitions", "retrieve few-shot examples")


review_graph.add_edge("retrieve few-shot examples", "classify emotion")
# 2) retrieve_few_shot_examples → classify_emotion

# 3) classify_emotion → route_review
review_graph.add_edge("classify emotion", "draft response")

#4 ) draft_response → notify_user
review_graph.add_edge("draft response", "notify user")

# 5) notify_user → END
review_graph.add_edge("notify user", END)


compiled_graph = review_graph.compile()

Print the graph

In [ ]:
Image(compiled_graph.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.API))

Testing the system

In [21]:
#generate test texts/ reviews to pass to my review bot
Example_text1 = "I love this product! It works great and exceeded my expectations."
Example_text2 = "I am very disappointed with the service. It was not what I expected."
Example_text3 = "The product is okay, but it could be better. I expected more."
Example_text4 = "the service was nothing special. I was hoping for a better experience. but product was good."


test if system is correctly categorizing the text and drafting a response & quality of the few-shot examples

In [34]:
# Process the legitimate email
print("\nProcessing review")
test_examples = compiled_graph.invoke(
    {
        "review_text": Example_text4, # invoke the graph with the email text
        "verbose": True,  # enable verbose output
    }
)


Processing review
Reviewbot received review: “the service was nothing special. I was hoping for a better experience. but product was good.”

review : the service was nothing special. I was hoping for a better experience. but product was good.
Review classified as: ['neutral']
Response:
Thank you for your feedback. We're glad to hear that you found the product good. We're sorry to hear that the service was not as exceptional as you hoped. We are constantly working to improve and would love to make things right. If there's anything specific you'd like us to address, please let us know.



Evaluating the system

In [ ]:
# Define emotion columns (ensure this matches your dataset and classify_emotion output)
emotion_columns = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

# 1. Load test data
try:
    df_test = pd.read_csv("data/ekman_test.csv")
    df_test = df_test[:50] #small dataset for testing the classification function
except FileNotFoundError:
    print("Error: data/ekman_test.csv not found. Make sure the path is correct.")
    raise

# Prepare lists for true and predicted labels
all_true_labels = []
all_pred_labels = []



checking split of different labels in the test data

In [ ]:

# Print count of each emotion in the test dataframe
print("\nEmotion distribution in the test set (df_test):")
for emotion in emotion_columns:
    if emotion in df_test.columns:
        count = df_test[emotion].sum()
        print(f"{emotion}: {count}")
    else:
        print(f"{emotion}: Column not found in df_test")


Emotion distribution in the test set (df_test):
anger: 74
disgust: 14
fear: 9
joy: 207
neutral: 140
sadness: 45
surprise: 64



Emotion distribution in the test set (df_test):
anger: 150
disgust: 25
fear: 26
joy: 412
neutral: 269
sadness: 93
surprise: 119


In [ ]:

print("Evaluating classify_emotion function...")

for index, row in tqdm(df_test.iterrows(), total=df_test.shape[0], desc="Processing test data"):
    text = row['text']
    
    # Prepare true labels for this sample
    true_emotions_for_sample = [col for col in emotion_columns if col in row and row[col] == 1]
    all_true_labels.append(true_emotions_for_sample)
    
    # Prepare state for classify_emotion
    # ReviewState is defined in a previous cell
    current_state = ReviewState(
        review_text=text,
        few_shot_examples=[], 
        emotion=None,         
        draft_reply=None,
        verbose=False         
    )
    
    try:
        retrived_state_update = fetch_emotion_definitions(current_state)
        current_state.update(retrived_state_update)
    except Exception as e:
        print(f"Error during fetch_emotion_definitions for text: '{text[:50]}...': {e}")
        all_pred_labels.append([]) # Append empty list for this sample on error
        continue

    try:
        retrieved_state_update = retrieve_few_shot_examples(current_state)
        current_state.update(retrieved_state_update)
    except Exception as e:
        print(f"Error during retrieve_few_shot_examples for text: '{text[:50]}...': {e}")
        all_pred_labels.append([]) # Append empty list for this sample on error
        continue

    # Call classify_emotion
    try:
        classification_result = classify_emotion(current_state)
        predicted_emotions_for_sample = classification_result.get('emotion', [])
        
        if isinstance(predicted_emotions_for_sample, str): # Ensure it's a list
            predicted_emotions_for_sample = [predicted_emotions_for_sample]
        
        all_pred_labels.append(predicted_emotions_for_sample)
    except Exception as e:
        print(f"Error during classify_emotion for text: '{text[:50]}...': {e}")
        all_pred_labels.append([]) # Append empty list for this sample on error
        continue

# 4. Evaluate
# Initialize MultiLabelBinarizer with all possible emotion classes
mlb = MultiLabelBinarizer(classes=emotion_columns)

# Fit on all possible labels to ensure consistent encoding, then transform
mlb.fit([emotion_columns]) # Fit with all known classes
y_true_binarized = mlb.transform(all_true_labels)
y_pred_binarized = mlb.transform(all_pred_labels)

# Print classification report
print("\nClassification Report for classify_emotion:")
report = classification_report(y_true_binarized, y_pred_binarized, target_names=mlb.classes_, zero_division=0)
print(report)

# Calculate overall metrics
precision_micro = precision_score(y_true_binarized, y_pred_binarized, average='micro', zero_division=0)
recall_micro = recall_score(y_true_binarized, y_pred_binarized, average='micro', zero_division=0)
f1_micro = f1_score(y_true_binarized, y_pred_binarized, average='micro', zero_division=0)

precision_macro = precision_score(y_true_binarized, y_pred_binarized, average='macro', zero_division=0)
recall_macro = recall_score(y_true_binarized, y_pred_binarized, average='macro', zero_division=0)
f1_macro = f1_score(y_true_binarized, y_pred_binarized, average='macro', zero_division=0)

precision_weighted = precision_score(y_true_binarized, y_pred_binarized, average='weighted', zero_division=0)
recall_weighted = recall_score(y_true_binarized, y_pred_binarized, average='weighted', zero_division=0)
f1_weighted = f1_score(y_true_binarized, y_pred_binarized, average='weighted', zero_division=0)


Evaluating classify_emotion function...


Processing test data: 100%|██████████| 50/50 [03:36<00:00,  4.33s/it]


Classification Report for classify_emotion:
              precision    recall  f1-score   support

       anger       0.14      0.20      0.17         5
     disgust       0.11      1.00      0.20         1
        fear       1.00      0.50      0.67         2
         joy       0.73      0.58      0.65        19
     neutral       0.57      0.72      0.63        18
     sadness       0.33      0.33      0.33         6
    surprise       0.33      0.60      0.43         5

   micro avg       0.46      0.57      0.51        56
   macro avg       0.46      0.56      0.44        56
weighted avg       0.55      0.57      0.54        56
 samples avg       0.46      0.60      0.50        56



In [42]:
# Print accuracy
accuracy = accuracy_score(y_true_binarized, y_pred_binarized)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.30


Testing the system on new self procured dataset of reviews

In [33]:

#defineing the columns
emotion_columns = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

# 1. Load new test data
new_test_csv_path = os.path.join("data", "reviews.csv")
try:
    df_reviews_test = pd.read_csv(new_test_csv_path, delimiter=';')
except FileNotFoundError:
    print(f"Error: {new_test_csv_path} not found. Make sure the path is correct.")
    raise
except Exception as e:
    print(f"Error loading {new_test_csv_path}: {e}")
    raise

In [ ]:
# Prepare lists for true and predicted labels
all_true_labels_reviews_csv = []
all_pred_labels_reviews_csv = []

print(f"\nEvaluating classify_emotion function on {new_test_csv_path}...")


for index, row in tqdm(df_reviews_test.iterrows(), total=df_reviews_test.shape[0], desc="Processing reviews.csv"):
    text = row['Text']
    
    # Prepare true labels for this sample
    # This assumes  CSV has columns named after emotions (e.g., 'joy', 'anger')
    true_emotions_for_sample = [col for col in emotion_columns if col in row and row[col] == 1]
    all_true_labels_reviews_csv.append(true_emotions_for_sample)
    
    # Prepare state for classify_emotion
    # ReviewState is defined in a previous cell
    current_state = ReviewState(
        review_text=text,
        few_shot_examples=[], 
        emotion=None,         
        draft_reply=None,
        verbose=False         
    )
    try:
        retrived_state_update = fetch_emotion_definitions(current_state)
        current_state.update(retrived_state_update)
    except Exception as e:
        print(f"Error during fetch_emotion_definitions for text: '{text[:50]}...': {e}")
        all_pred_labels.append([]) # Append empty list for this sample on error
        continue

    # Call retrieve_few_shot_examples (defined in cell id: 0bc7bdc9)
    try:
        retrieved_state_update = retrieve_few_shot_examples(current_state)
        current_state.update(retrieved_state_update) # Update the state directly
    except Exception as e:
        print(f"Error during retrieve_few_shot_examples for text: '{text[:50]}...': {e}")
        all_pred_labels_reviews_csv.append([]) # Append empty list for this sample on error
        continue

    # Call classify_emotion (defined in cell id: 0bc7bdc9)
    try:
        classification_result = classify_emotion(current_state)
        predicted_emotions_for_sample = classification_result.get('emotion', [])
        
        if isinstance(predicted_emotions_for_sample, str): # Ensure it's a list
            predicted_emotions_for_sample = [predicted_emotions_for_sample]
        
        all_pred_labels_reviews_csv.append(predicted_emotions_for_sample)
    except Exception as e:
        print(f"Error during classify_emotion for text: '{text[:50]}...': {e}")
        all_pred_labels_reviews_csv.append([]) # Append empty list for this sample on error
        continue

# 4. Evaluate
mlb_reviews_csv = MultiLabelBinarizer(classes=emotion_columns)

# Fit on all possible labels to ensure consistent encoding, then transform
mlb_reviews_csv.fit([emotion_columns]) 
y_true_binarized_reviews_csv = mlb_reviews_csv.transform(all_true_labels_reviews_csv)
y_pred_binarized_reviews_csv = mlb_reviews_csv.transform(all_pred_labels_reviews_csv)

# Print classification report
print(f"\nClassification Report for classify_emotion on {new_test_csv_path}:")
report_reviews_csv = classification_report(y_true_binarized_reviews_csv, y_pred_binarized_reviews_csv, target_names=mlb_reviews_csv.classes_, zero_division=0)
print(report_reviews_csv)

# Calculate overall metrics
precision_micro_reviews_csv = precision_score(y_true_binarized_reviews_csv, y_pred_binarized_reviews_csv, average='micro', zero_division=0)
recall_micro_reviews_csv = recall_score(y_true_binarized_reviews_csv, y_pred_binarized_reviews_csv, average='micro', zero_division=0)
f1_micro_reviews_csv = f1_score(y_true_binarized_reviews_csv, y_pred_binarized_reviews_csv, average='micro', zero_division=0)

precision_macro_reviews_csv = precision_score(y_true_binarized_reviews_csv, y_pred_binarized_reviews_csv, average='macro', zero_division=0)
recall_macro_reviews_csv = recall_score(y_true_binarized_reviews_csv, y_pred_binarized_reviews_csv, average='macro', zero_division=0)
f1_macro_reviews_csv = f1_score(y_true_binarized_reviews_csv, y_pred_binarized_reviews_csv, average='macro', zero_division=0)

precision_weighted_reviews_csv = precision_score(y_true_binarized_reviews_csv, y_pred_binarized_reviews_csv, average='weighted', zero_division=0)
recall_weighted_reviews_csv = recall_score(y_true_binarized_reviews_csv, y_pred_binarized_reviews_csv, average='weighted', zero_division=0)
f1_weighted_reviews_csv = f1_score(y_true_binarized_reviews_csv, y_pred_binarized_reviews_csv, average='weighted', zero_division=0)



Evaluating classify_emotion function on data\reviews.csv...


Processing reviews.csv: 100%|██████████| 79/79 [03:17<00:00,  2.50s/it]


Classification Report for classify_emotion on data\reviews.csv:
              precision    recall  f1-score   support

       anger       0.83      0.80      0.82        25
     disgust       0.33      0.83      0.48         6
        fear       0.00      0.00      0.00         2
         joy       0.86      1.00      0.92        36
     neutral       0.86      0.50      0.63        12
     sadness       0.44      0.73      0.55        11
    surprise       0.33      0.20      0.25        10

   micro avg       0.69      0.75      0.72       102
   macro avg       0.52      0.58      0.52       102
weighted avg       0.71      0.75      0.71       102
 samples avg       0.73      0.79      0.73       102

